# Klasifikasi Limbah

------

## Data Cleaning

In [1]:
import os
import numpy as np
import pandas as pd
import lxml.html as lh

In [2]:
path = "/Users/bunny/myGit/DataStore/Limbah/"
table = ""

for ls in os.listdir(path):
    if ls.endswith(".html"):
        print (ls)
        with open(path + ls, 'r') as file:
            table = file.read()

data-limbah.html


In [3]:
doc = lh.fromstring(table)
tr = doc.xpath("//tr")

In [4]:
feature_name = []

for td in tr[0]:
    feature_name.append(td.text_content())

feature_name = [feature_name[2]] + feature_name[13:16] + [feature_name[17]]

In [5]:
data = []

for i in range(1, len(tr[1:])):
    tr_ = tr[i]
    ccol = 0
    data_tmp = []
    
    for td in tr_:
        if ccol == 2 or (ccol >= 13 and ccol <= 17) and ccol != 16:
            data_tmp.append(td.text_content())
        ccol += 1
    
    data.append(data_tmp)

In [6]:
len(data)

10269

In [7]:
df = pd.DataFrame(data, columns=feature_name)

In [8]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 5 columns):
Nama Pengirim      1020 non-null object
Kode Limbah        1020 non-null object
Jenis Limbah       1020 non-null object
Karakter Limbah    1020 non-null object
Kemasan            1020 non-null object
dtypes: object(5)
memory usage: 39.9+ KB


In [9]:
df = df.drop(["Nama Pengirim"], axis=1)

In [10]:
df.sample(3)

,Kode Limbah,Jenis Limbah,Karakter Limbah,Kemasan
307,B328-4,Printed circuit board (PCB),Beracun (toxic - T),Tanpa Kemasan
683,B107d,Limbah elektronik termasuk cathode ray tube (C...,Beracun (toxic - T),Drum Plastik
481,A338-3,Residu sampel Limbah B3,Cairan Mudah menyala (ignitable - I),Kemasan Karton atau plastik


In [11]:
kode = pd.read_csv("./Kode/kode-limbah.csv")
kode.sample(3)

,Kode,Sumber,Bahaya
663,B329-4,Spesifik Umum,2
181,A2118,?,1
224,A2161,?,1


In [12]:
bentuk = pd.read_csv("./Kode/bentuk.csv")
bentuk.sample(3)

,Jenis,Bentuk
57,Bahan atau produk yang tidak memenuhi spesifik...,Cair & Padat
20,Larutan asam (pickling),Cair
39,Debu fasilitas pengendalian pencemaran udara.,Padat


In [13]:
df = df.join(kode.set_index("Kode"), on="Kode Limbah")
df = df.join(bentuk.set_index("Jenis"), on="Jenis Limbah")

In [14]:
df = df[["Kode Limbah", "Bentuk", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]
# df = df[["Kode Limbah", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]

In [15]:
df = df.astype("category")

## Data Exploration

In [16]:
df.sample(3)

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
765,B109d,Padat,Beracun (toxic - T),Tidak Spesifik,2,"Karung Kain, plastik, kertas"
399,A318-4,Padat,Beracun (toxic - T),Spesifik Umum,1,Drum Logam
504,B327-5,Padat,Beracun (toxic - T),Spesifik Umum,2,R.O Container


In [17]:
df["Bentuk"].value_counts()

Padat           688
Cair            319
Cair & Padat     13
Name: Bentuk, dtype: int64

In [18]:
df["Karakter Limbah"].value_counts()

Beracun (toxic - T)                      805
Cairan Mudah menyala (ignitable - I)     100
Infeksius (infectious - X)                42
Padatan Mudah menyala (ignitable - I)     35
Korosif (corrosive - C)                   33
Padatan Mudah meledak (explosive – E)      4
Cairan Mudah meledak (explosive – E)       1
Name: Karakter Limbah, dtype: int64

In [19]:
df["Sumber"].value_counts()

Tidak Spesifik     534
Spesifik Umum      374
Spesifik Khusus    111
?                    1
Name: Sumber, dtype: int64

In [20]:
df["Bahaya"].value_counts()

2    631
1    389
Name: Bahaya, dtype: int64

In [21]:
df["Kemasan"].value_counts()

Drum Logam                      287
Tanpa Kemasan                   218
Jumbo Bag                       153
Kemasan Karton atau plastik     122
Box Besi                         51
Tangki                           40
Karung Kain, plastik, kertas     39
Drum Plastik                     34
IBC Tank                         26
R.O Container                    20
Jerycan                          12
Wheel Bin                        10
Drum kayu                         5
Wadah Hopper                      1
Gerbong Barang                    1
                                  1
Name: Kemasan, dtype: int64

### Outlier

In [22]:
df[df["Sumber"] == '?']

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
800,A2358,Cair,Beracun (toxic - T),?,1,Drum Logam


In [23]:
df[df["Kemasan"] == ""]

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
982,B105d,Cair,Cairan Mudah menyala (ignitable - I),Tidak Spesifik,2,
